In [1]:
import pandas as pd
import pydicom
from operator import itemgetter
from scipy.stats import gaussian_kde
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.stats import norm
from sklearn import preprocessing

In [2]:
train_df = pd.read_csv('../data/train_2016_v2.csv',index_col=False, parse_dates=['transactiondate'])
properties_df = pd.read_csv('../data/properties_2016.csv')
properties_df = properties_df.fillna(0)
train_df = train_df.fillna(0)

In [3]:
meansd=abs(-0.16107794320832886/2+0.011457219606756682) # use this based on exprience to make lowerror number and high error number dont have huge difference.
train_df['class'] = pd.cut(abs(train_df.logerror),[0,meansd,1e6],3,
                                 labels=['lowerror','higherror']) # this creates a new variable
train_df['plotclass'] = pd.cut(abs(train_df.logerror),[0,meansd,1e6],3,
                                 labels=['0','1'])
train_df['class'] = pd.cut(abs(train_df.logerror),[0,meansd,1e6],3,
                                 labels=['lowerror','higherror']) # this creates a new variable

In [5]:
#Switching the order of the dataframes on the merge changes the outcome in the Random Forest
# mergelogerror_properties = pd.merge(properties_df,train_df, on='parcelid')
train_merge_df = pd.merge(properties_df,train_df, on='parcelid')
#train_merge_df.fillna(0)

In [6]:
df= train_merge_df[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet','logerror']].copy()
df_scaled = preprocessing.scale(df[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet','logerror']])
df_scaled=pd.DataFrame(df_scaled)
df_scaled.columns=[['longitude','latitude','fullbathcnt','calculatedbathnbr','buildingqualitytypeid','heatingorsystemtypeid','yearbuilt','taxamount','lotsizesquarefeet','logerror']]


In [7]:
y=train_df['logerror'].values.astype(np.float32)
print(y)

[ 0.0276     -0.1684     -0.004      ..., -0.26789999  0.0602      0.42070001]


In [8]:
train_df.head()

,parcelid,logerror,transactiondate,class,plotclass
0,11016594,0.0276,2016-01-01,lowerror,0
1,14366692,-0.1684,2016-01-01,higherror,1
2,12098116,-0.0040,2016-01-01,lowerror,0
3,12643413,0.0218,2016-01-02,lowerror,0
4,14432541,-0.0050,2016-01-02,lowerror,0


In [9]:
# CLEANED DATASET
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import NMF
from  xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

#Random Forest
X_train, X_test, Y_train, Y_test = train_test_split(df_scaled, y, random_state=0)

rf = RandomForestRegressor(n_estimators=30,max_depth=100,random_state=42)#,criterion='mae')
rf.fit(X_train, Y_train)
y_pred = rf.predict(X_test)
df_y=abs(Y_test-y_pred)
MAE=df_y.sum()/len(Y_test)
print('Mean Absolute Error:', MAE)

Mean Absolute Error: 0.079395083111


ValueError: fill value must be in categories

In [15]:
# FULL DATASET
from sklearn.decomposition import PCA
from sklearn.decomposition.pca import PCA
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

#PCA
train_merge_df_2 = train_merge_df.drop(['propertycountylandusecode','propertyzoningdesc','taxdelinquencyflag','class','plotclass'],axis = 1)

train_merge_df_2['transactiondate']=train_merge_df_2['transactiondate'].dt.month
#train_merge_df_2=train_merge_df_2.dropna()
train_merge_df_2 = StandardScaler().fit_transform(train_merge_df_2)

pca=PCA(n_components= 22)
train_features_2 = pca.fit_transform(train_merge_df_2)
pca.explained_variance_ratio_.sum()
print('Total Explained Variance:',pca.explained_variance_ratio_.sum())

Total Explained Variance: 0.815576620386


In [16]:
#Random Forest
X_train, X_test, Y_train, Y_test = train_test_split(train_features_2, y, random_state=0)

rf = RandomForestRegressor(n_estimators=100,max_depth=100,random_state=42)#,criterion='mae')
rf.fit(X_train, Y_train)
y_pred = rf.predict(X_test)
df_y=abs(Y_test-y_pred)
MAE=df_y.sum()/len(Y_test)
print('Mean Absolute Error:', MAE)

Mean Absolute Error: 0.0744946903059
